In [1]:
import json
import logging
import os
import sys
from os import path
import warnings
warnings.filterwarnings("ignore")
from cli.cli_parser import parse_args
from graph.graph import AssociatedGraph
from utils.preprocessing import create_graphs
import pandas as pd

In [2]:
class obj:
    def __init__(self, dict1):
        self.__dict__.update(dict1)
 
def dict2obj(dict1):
    return json.loads(json.dumps(dict1), object_hook=obj)

In [3]:
args = dict2obj({
        "folder_path": "Analysis/selected_strs_renumber/without_TCR",
        "residues_lists": "jsons/residues_lists.json",
        "serd_config": None,
        "files_name": None,
        "output_path": None,
        "run_name": None,
        "check_depth": False,
        "check_rsa": True,
        "centroid_threshold": 8.5,
        "distance_diff_threshold": 3,
        "depth_filter": None,
        "depth_bins": 3,
        "rsa_filter": 0.1,
        "rsa_bins": 3,
        "distance_bins": 3,
        "factors_path": "resources/atchley_aa.csv",
        "classes_path": None})

checks = {
        "depth": args.check_depth,
        "rsa": args.check_rsa,
}

association_config = {
    "centroid_threshold": args.centroid_threshold,
    "distance_diff_threshold": args.distance_diff_threshold,
    "rsa_filter": args.rsa_filter,
    "depth_filter": args.depth_filter,
    "rsa_bins": args.rsa_bins,
    "depth_bins": args.depth_bins,
    "distance_bins": args.distance_bins,
    "checks": checks,
    "factors_path": args.factors_path,
    "classes_path": args.classes_path
}



In [4]:
diffMHC_diffPep = pd.read_csv("Analysis/crossreact_processed_diff_MHC_diff_pep_helder.csv")
diffMHC_SamePep = pd.read_csv("Analysis/crossreact_processed_diff_MHC_same_pep_helder.csv")
sameMHC_diffPep = pd.read_csv("Analysis/crossreact_processed_same_MHC_diff_pep_helder.csv")
rawCross = pd.read_csv("Analysis/crossreact_tcrs_dedup_noncanonicalclassv2_samuel.csv")

In [5]:
rawCross

,TCR_ID,TRA,TRB,peptide,MHCseq,MHCseq_ref,allele,Score,Release_date,Resolution,NonCanonical,Reference,TCR_pair_id,MHC_allele_id,peptide_id,pMHC_id,pep_crossreact,mhc_crossreact,pmhc_crossreact
0,PDB4ms8,AQSVTQPDARVTVSEGASLQLRCKYSYSATPYLFWYVQYPRQGLQM...,EAAVTQSPRNKVTVTGGNVTLSCRQTNSHNYMYWYRQDTGHGLRLI...,SPAEAGFFL,MGPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRY...,MGPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRY...,NaN,3,2014-09-24,1.92,NaN,4MS8,0,0,3,3,True,False,True
1,PDB3tjh,AQSVTQPDARVTVSEGASLQLRCKYSYSATPYLFWYVQYPRQGLQM...,EAAVTQSPRNKVTVTGGNVTLSCRQTNSHNYMYWYRQDTGHGLRLI...,SPLDSLWWI,MGPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRY...,GPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRYE...,NaN,3,2011-12-07,2.12,NaN,3TJH,0,6,11,11,True,False,True
2,PDB4mxq,AQSVTQPDARVTVSEGASLQLRCKYSYSATPYLFWYVQYPRQGLQM...,EAAVTQSPRNKVTVTGGNVTLSCRQTNSHNYMYWYRQDTGHGLRLI...,SPAPRPLDL,MGPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRY...,MGPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRY...,NaN,3,2015-08-19,2.60,NaN,4MXQ,0,0,15,16,True,False,True
3,PDB3tfk,AQSVTQPDARVTVSEGASLQLRCKYSYSATPYLFWYVQYPRQGLQM...,EAAVTQSPRNKVTVTGGNVTLSCRQTNSHNYMYWYRQDTGHGLRLI...,QLSDVPMDL,MGPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRY...,GPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRYE...,NaN,3,2011-12-07,2.75,NaN,3TFK,0,6,78,86,True,False,True
4,PDB4n0c,AQSVTQPDARVTVSEGASLQLRCKYSYSATPYLFWYVQYPRQGLQM...,EAAVTQSPRNKVTVTGGNVTLSCRQTNSHNYMYWYRQDTGHGLRLI...,MPAGRPWDL,MGPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRY...,MGPHSMRYYETATSRRGLGEPRYTSVGYVDDKEFVRFDSDAENPRY...,NaN,3,2015-08-19,2.90,NaN,4N0C,0,0,51,55,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,PDB8v50,QSLEQPSEVTAVEGAIVQINCTYQTSGFYGLSWYQQHDGGAPTFLS...,GVSQSPRYKVAKRGQDVALRCDPISGHVSLFWYQQALGQGPEFLTY...,LPFDKSTVM,SHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTEP...,MRVTAPRTVLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,B*35:01:01:01,3,2024-12-18,2.65,NaN,8V50,30,13,66,72,True,False,False
93,PDB7n2o,KQEVTQIPAALSVPEGENLVLNCSFTDSAIYNLQWFRQDPGKGLTS...,GVTQTPKHLITATGQRVTLRCSPRSGDLSVYWYQQSLDQGLQFLIQ...,LRVMMLAPF,GSHSMRYFHTSVSRPGRGEPRFITVGYVDDTLFVRFDSDAASPREE...,GSHSMRYFHTSVSRPGRGEPRFITVGYVDDTLFVRFDSDAASPREE...,NaN,3,2022-12-07,2.30,NaN,7N2O,31,23,72,80,True,False,False
94,PDB7n2n,KQEVTQIPAALSVPEGENLVLNCSFTDSAIYNLQWFRQDPGKGLTS...,GVTQTPKHLITATGQRVTLRCSPRSGDLSVYWYQQSLDQGLQFLIQ...,TRLALIAPK,GSHSMRYFHTSVSRPGRGEPRFITVGYVDDTLFVRFDSDAASPREE...,GSHSMRYFHTSVSRPGRGEPRFITVGYVDDTLFVRFDSDAASPREE...,NaN,3,2022-12-07,2.60,NaN,7N2N,31,23,53,57,True,False,False
95,PDB8eo8,GENVEQHPSTLSVQEGDSAVIKCTYSDSASNYFPWYKQELGKRPQL...,AVVSQHPSRVICKSGTSVKIECRSLDFQATTMFWYRQFPKQSLMLM...,LPFDKATIM,GSHSMRYFYTAMSRPGRGEPRFIAVGYVDDTQFVRFDSDAASPRTE...,MRVTAPRTVLLLLWGAVALTETWAGSHSMRYFYTAMSRPGRGEPRF...,B*35:01:01:01,3,2024-03-27,2.30,NaN,8EO8,32,13,76,84,True,False,False


In [6]:
crossDf = rawCross[["TCR_ID", "TCR_pair_id", "MHC_allele_id", "peptide_id", "pMHC_id", "Reference"]]
crossDf

,TCR_ID,TCR_pair_id,MHC_allele_id,peptide_id,pMHC_id,Reference
0,PDB4ms8,0,0,3,3,4MS8
1,PDB3tjh,0,6,11,11,3TJH
2,PDB4mxq,0,0,15,16,4MXQ
3,PDB3tfk,0,6,78,86,3TFK
4,PDB4n0c,0,0,51,55,4N0C
...,...,...,...,...,...,...
92,PDB8v50,30,13,66,72,8V50
93,PDB7n2o,31,23,72,80,7N2O
94,PDB7n2n,31,23,53,57,7N2N
95,PDB8eo8,32,13,76,84,8EO8


In [7]:
print(crossDf["Reference"][4])

4N0C


In [8]:
for k, v in TCR_pair_id:
    print(k, v)

NameError: name 'TCR_pair_id' is not defined

In [9]:
TCR_pair_id = crossDf.groupby('TCR_pair_id')
Graphs = {}
for pair_id, group in TCR_pair_id:
    print(f"\n--- TCR_pair_id: {pair_id} ---")
    args.files_name = ",".join([f"noTCR_{name.lower()}.trunc.fit_renum.pdb" for name in group["Reference"]])
    args.run_name = pair_id
    args.output_path = f"Analysis/CrossGraphs/{pair_id}/All"

    graphs = create_graphs(args)
    try:
        G = AssociatedGraph(
            graphs=graphs,
            output_path=args.output_path,
            run_name=args.run_name,
            association_config=association_config
        )
    except Exception as e:
        print(f"Não foi possível criar grafo: {e}")
    else:
        G.draw_graph(show=False, save=True)
        Graphs[pair_id] = G
        graph_data = dict()
        for j, comps in enumerate(G.associated_graphs):
            graph_data[j] = {"comp": j, "frames": {}}
            for i in range(len(comps[0])):
                nodes = list(comps[0][i].nodes)
                edges = list(comps[0][i].edges)
                neighbors = {
                    str(node): [str(neighbor) for neighbor in comps[0][i].neighbors(node)]
                    for node in nodes     
                    }

                graph_data[j]["frames"][i] = {
                    "nodes": nodes,
                    "edges": edges,
                    "neighbors": neighbors
                }

        output_json = path.join(args.output_path, f"graph_{args.run_name}.json")
        with open(output_json, "w") as f:
            json.dump(graph_data, f, indent=4)
        print(f"Graph data saved to {output_json}")



Output()


--- TCR_pair_id: 0 ---


Output()

Subgraph exposed_residues created with success!


Output()

...

Graph data saved to Analysis/CrossGraphs/31/All/graph_31.json

--- TCR_pair_id: 32 ---


Output()

Subgraph exposed_residues created with success!


Subgraph exposed_residues created with success!
Graph data saved to Analysis/CrossGraphs/32/All/graph_32.json


<Figure size 640x480 with 0 Axes>

In [10]:
Graphs

{0: <graph.graph.AssociatedGraph at 0x7efc55cb6070>,
 1: <graph.graph.AssociatedGraph at 0x7efc548da310>,
 2: <graph.graph.AssociatedGraph at 0x7efc54b4cc70>,
 3: <graph.graph.AssociatedGraph at 0x7ef75ea344c0>,
 4: <graph.graph.AssociatedGraph at 0x7efc54fd2520>,
 5: <graph.graph.AssociatedGraph at 0x7efc54b3e070>,
 6: <graph.graph.AssociatedGraph at 0x7efc54a38a90>,
 7: <graph.graph.AssociatedGraph at 0x7efc32f4c940>,
 8: <graph.graph.AssociatedGraph at 0x7efc32f4c730>,
 9: <graph.graph.AssociatedGraph at 0x7ef75eb89a60>,
 10: <graph.graph.AssociatedGraph at 0x7efc550196d0>,
 11: <graph.graph.AssociatedGraph at 0x7efc55cb6490>,
 12: <graph.graph.AssociatedGraph at 0x7efc55800730>,
 13: <graph.graph.AssociatedGraph at 0x7efc19c3c3d0>,
 14: <graph.graph.AssociatedGraph at 0x7efc4405ef10>,
 15: <graph.graph.AssociatedGraph at 0x7efc334549d0>,
 16: <graph.graph.AssociatedGraph at 0x7efc32e845b0>,
 17: <graph.graph.AssociatedGraph at 0x7efc553baeb0>,
 19: <graph.graph.AssociatedGraph at 0

In [ ]:
d = {"teste":4}
d.teste

AttributeError: 'dict' object has no attribute 'teste'